In [1]:
# Libraries for Feature Engineering
import seaborn as sns

In [2]:
# Libraries for Data Structures
import pandas as pd
import numpy as np

In [3]:
# Libraries for Creating Pipelines
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [4]:
# Libraries for Data Pre-processing and Processing
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [5]:
# Libraries For Model Formation
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.neural_network import MLPClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import linear_model
# import xgboost as xgb
# from xgboost import XGBClassifier
from sklearn.cluster import KMeans
# from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [6]:
# Libraries for Model Evaluation
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import accuracy_score, classification_report


# Dataset Customisation

In [7]:
# Importing Dataset from csv file
raw_data = pd.read_csv('../Datasets/alzheimers.csv')

# Gives total no.of rows and columns
raw_data.shape

(373, 15)

In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Subject ID  373 non-null    object 
 1   MRI ID      373 non-null    object 
 2   Group       373 non-null    object 
 3   Visit       373 non-null    int64  
 4   MR Delay    373 non-null    int64  
 5   M/F         373 non-null    object 
 6   Hand        373 non-null    object 
 7   Age         373 non-null    int64  
 8   EDUC        373 non-null    int64  
 9   SES         354 non-null    float64
 10  MMSE        371 non-null    float64
 11  CDR         373 non-null    float64
 12  eTIV        373 non-null    int64  
 13  nWBV        373 non-null    float64
 14  ASF         373 non-null    float64
dtypes: float64(5), int64(5), object(5)
memory usage: 43.8+ KB


## ***DataSet Columns Defination***

```
1. Subject ID: Unique identifier for each subject in the dataset.
2. MRI ID: Unique identifier for each MRI scan.
3. Group: Represents the group or condition of the subject (e.g., 'AD' for Alzheimer's Disease).
4. Visit: Number indicating the visit sequence for a particular subject.
5. MR Delay: Time delay between consecutive MRI scans for a subject.
6. M/F: Gender of the subject (Male or Female).
7. Hand: Handedness of the subject (e.g., 'R' for right-handed).
8. Age: Age of the subject.
9. EDUC: Years of education completed by the subject.
10. SES: Socioeconomic status of the subject.
11. MMSE: Mini-Mental State Examination score, a measure of cognitive function.
12. CDR: Clinical Dementia Rating, a measure of dementia severity.
13. eTIV: Estimated total intracranial volume.
14. nWBV: Normalized whole brain volume.
15. ASF: Atlas scaling factor, a measure related to brain size normalization.
    
```



In [9]:
new_column_names = ['Id','MRI_ID', 'Target_Variable', 'Visit_Counts', 'MRI_Scan_Delay','Gender', 'Dominant_Hand', 'Age', 'Education', 'Socioeconomic_Status', 'Mini_Mental_State_Examination', 'Clinical_Dementia_Rating', 'Estimated_Total_Intracranial_Volume', 'Normalize_Whole_Brain_Volume', 'Atlas_Scaling_Factor']
raw_data.rename(columns={old_column_name: new_column_name for old_column_name, new_column_name in zip(raw_data.columns, new_column_names)}, inplace=True)

# Feature Engineering

## Dropping Non-Required Featues

In [10]:
raw_data.drop(['Id', 'MRI_ID','Gender', 'Dominant_Hand'], axis=1, inplace=True)

## Converting Category into Numerical Values

In [11]:
# Converting Target Variables to Binary Target Values
raw_data.Target_Variable.replace(['Demented','Converted'], 1, inplace=True)
raw_data.Target_Variable.replace('Nondemented', 0, inplace=True)
raw_data['Target_Variable'].value_counts()

Target_Variable
0    190
1    183
Name: count, dtype: int64

In [12]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Target_Variable                      373 non-null    int64  
 1   Visit_Counts                         373 non-null    int64  
 2   MRI_Scan_Delay                       373 non-null    int64  
 3   Age                                  373 non-null    int64  
 4   Education                            373 non-null    int64  
 5   Socioeconomic_Status                 354 non-null    float64
 6   Mini_Mental_State_Examination        371 non-null    float64
 7   Clinical_Dementia_Rating             373 non-null    float64
 8   Estimated_Total_Intracranial_Volume  373 non-null    int64  
 9   Normalize_Whole_Brain_Volume         373 non-null    float64
 10  Atlas_Scaling_Factor                 373 non-null    float64
dtypes: float64(5), int64(6)
memory u

# Dataset Pre-Processing and Processing

In [13]:
# Separate labels and features
X = raw_data.drop(['Target_Variable'], axis=1)
y = raw_data['Target_Variable']


# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Impute missing values
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X_scaled)

# Data Augmentation
smote = SMOTE(random_state=42)
X_imputed_smote, y_smote = smote.fit_resample(X_imputed, y)

# Now, split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed_smote, y_smote, test_size=0.2, random_state=42)

# Model Formations

In [14]:
# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
rf_grid = GridSearchCV(rf_model, rf_params, cv=5)
rf_grid.fit(X_train, y_train)
best_rf_model = rf_grid.best_estimator_

cv_scores = cross_val_score(best_rf_model, X_train, y_train, cv=5)
print(f"{best_rf_model} CV Accuracy: {np.mean(cv_scores) * 100:.2f}%")

# Fit the model on the entire dataset
best_rf_model.fit(X_train, y_train)
y_pred = best_rf_model.predict(X_test)

print("-------------------------------------------------------")
# Training accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"{best_rf_model} Training Accuracy: {accuracy * 100:.2f}%")
print("-------------------------------------------------------")

# Classification report
print(f"{best_rf_model} Classification Report:\n{classification_report(y_test, y_pred)}")
print("-------------------------------------------------------")

RandomForestClassifier(random_state=42) CV Accuracy: 95.07%
-------------------------------------------------------
RandomForestClassifier(random_state=42) Training Accuracy: 93.42%
-------------------------------------------------------
RandomForestClassifier(random_state=42) Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93        34
           1       0.97      0.90      0.94        42

    accuracy                           0.93        76
   macro avg       0.93      0.94      0.93        76
weighted avg       0.94      0.93      0.93        76

-------------------------------------------------------


In [15]:
import pickle


filename = "alzheimer.sav"
pickle.dump(best_rf_model, open(filename, "wb"))

In [16]:
pickle.load(open(filename, "rb"))

RandomForestClassifier(random_state=42)